<img src="ariel.png" alt="Image 1" width="150" style="float: left; margin-right: 20px;">
<img src="cyber.png" alt="Image 2" width="150" style="float: left;">


# ModelGuard: disarm and reconstruction
### Raz Elbaz
### Dr. Ran Dubin and Prof. Amit Dvir

#### Student ID: 207276775





In [44]:
import socket
import sys
import unittest
from unittest.mock import patch
import fickling.analysis as analysis
from fickling.pickle import Pickled
import builtins
import os
from IPython.display import Markdown
import pickle
import ast
import pickletools
from typing import Optional, TextIO, Tuple

import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential


In [45]:
if sys.version_info < (3, 9):
    from astunparse import unparse
else:
    from ast import unparse

from fickling.pickle import Interpreter, Pickled
class cdr:
    def __init__(self):
        pass

    def code(self, pickled: Pickled) -> str:
        """
        Returns the string representation of the code object that was pickled.
        """
        code = pickled['code']
        return code.co_code.decode('utf-8')

    def check_safety(
        self, pickled: Pickled, filename, stdout: Optional[TextIO] = None, stderr: Optional[TextIO] = None
    ) -> bool:
        if stdout is None:
            stdout = sys.stdout
        if stderr is None:
            stderr = sys.stderr

        properties = pickled.properties
        likely_safe = True
        reported_shortened_code = set()

        def shorten_code(ast_node) -> Tuple[str, bool]:
            code = unparse(ast_node).strip()
            if len(code) > 32:
                cutoff = code.find("(")
                if code[cutoff] == "(":
                    shortened_code = f"{code[:code.find('(')].strip()}(...)"
                else:
                    shortened_code = code
            else:
                shortened_code = code
            was_already_reported = shortened_code in reported_shortened_code
            reported_shortened_code.add(shortened_code)
            return shortened_code, was_already_reported

        safe_lines = []

        with open(filename, 'rb') as f:
            code = str(f.read().decode('latin1'))

        for line in code.split('\n'):
            try:
                clean_string = line.replace('\x00', '')
                ast_node = compile(clean_string, '<string>', 'exec', ast.PyCF_ONLY_AST)
            except SyntaxError:
                continue
            is_safe = True
            for node in ast.walk(ast_node):
                if isinstance(node, ast.Call):
                    if (
                        isinstance(node.func, ast.Name)
                        and node.func.id == 'eval'
                    ):
                        is_safe = False
                    elif (
                        isinstance(node.func, ast.Attribute)
                        and node.func.attr == 'loads'
                        and isinstance(node.func.value, ast.Name)
                        and node.func.value.id == 'pickle'
                    ):
                        is_safe = False
                elif isinstance(node, ast.Import):
                    for alias in node.names:
                        if not alias.name.startswith('_') and alias.name not in sys.modules:
                            is_safe = False
                elif isinstance(node, ast.ImportFrom):
                    if not node.module.startswith('_') and node.module not in sys.modules:
                        is_safe = False
                elif ("eval" in line) or ("exec" in line) or ("compile" in line) or ("open" in line):
                    is_safe = False
                elif ("__builtin__"in line) or ("os" in line) or ("subprocess" in line) or ("sys" in line) or ("builtins" in line) or ("socket" in line):
                    is_safe = False

        with open(filename, 'rb') as f:
            data = f.read()

        for op in pickletools.genops(data):
            if type(op[1]) == str and all(substring not in op[1] for substring in ["eval", "exec", "compile", "open", "__builtin__", "os", "subprocess", "sys", "builtins", "socket"]):
                safe_lines.append(op[1])

        with open(filename, 'wb') as f:
            pickle.dump('\n'.join(safe_lines), f)

        if not safe_lines:
            return "False"

        if likely_safe:
            return "True"
        else:
            return "False"


In [46]:
def process_file(filename):
    display(Markdown("-------------------------- " + os.path.splitext(filename)[0] + " ----------------------------------"))
    
    with patch('sys.stdout') as stdout:
            
        with open(filename, 'rb') as f:
                pickled_data = f.read()
        pickled_obj = Pickled.load(pickled_data)
            # First run analysis.py
        analysis_result = analysis.check_safety(pickled_obj)
        display(str(analysis_result)) # Expecting clean
        if str(analysis_result) == '':
            display(Markdown("clean"))
        else:
            display(Markdown("not clean"))
            scan_pickle_file.scann(filename)
            display(Markdown("Now removing the malicious data...."))
            with patch('sys.stdout') as stdout:
                cdr().check_safety(pickled_obj,filename)
                
                # Finally, run analysis.py again
                with open(filename, 'rb') as f:
                    pickled_data = f.read()
                pickled_obj = Pickled.load(pickled_data)
                analysis_result_2 = analysis.check_safety(pickled_obj)
                display(str(analysis_result_2)) # Expecting clean
            # Check stdout for expected messages
            if str(analysis_result_2) == '':
                display(Markdown("clean"))
                display(Markdown("\nThe clean data left in the file:"))
                with open(filename, 'rb') as f:
                    pickled_data = pickle.load(f)
                display(str(pickled_data))
            else:
                display(Markdown("not clean"))

In [47]:
import ast
import json
import os
import re
import subprocess
import sys
from fickling.pickle import Pickled
import pickle
import _pickle as cPickle

from termcolor import colored
# -attacks
# https://www.cadosecurity.com/linux-attack-techniques-dynamic-linker-hijacking-with-ld-preload/
# https://www.cybertriage.com/blog/training/how-to-detect-running-malware-intro-to-incident-response-triage-part-7/

# https://www.beyondtrust.com/blog/entry/important-linux-files-protect
BAD_LIBRARY = {'/etc/hosts', '/bin/sh', '/etc/passwd', '/etc/pam.conf', '/proc', '/etc/shadow', '/etc/profile',
               '~/.bash_profile', '~/.bash_login', '~/.profile. /home/user/.bashrc', '/etc/bash.bashrc',
               '/etc/profile.d/', '/etc/system.d', '/etc/rc.*', '/etc/init.*.', '/etc/resolv.conf', '/etc/gshadow',
               '/etc/pam.d', '/bin', '/sbin'}
# This technique is often called DLL injection on Windows.
# With DLL injection, the attacker creates a malicious library with the same name and API as the good one.
# The program loads the malicious library and it, in turn, loads the good one and it will call the good one as needed to do the operations that the original program wants.
BAD_CALLS = {'os', 'shutil', 'sys', 'requests', 'net', 'func',
             'args',
             'keywords', }
BAD_SIGNAL = {'eval', 'compile', 'rm ', 'cat ', 'nc ', 'exec', 'open', 'run'}
BAD_FILES = {'.py', '.exe', '.dll', '.so'}
# https://redcanary.com/threat-detection-report/techniques/powershell/
# PowerShell -encodedcommand switch
# This detection analytic looks for the execution of powershell.exe with command lines that include variations of the -encodedcommand argument; PowerShell will recognize and accept anything from -e onward, and it will show up outside of the encoded bits.
BAD_COMMAND = {'powershell.exe', '-e', '-en', '-enc', '-enco', 'ls', 'base64'}
# Obfuscation and escape characters
# Obfuscation can disrupt detection logic by splitting commands or parameters or inserting extra characters (that are ignored by PowerShell).
# Monitor for the execution of PowerShell with unusually high counts of characters like ^, +, $, and %.
BAD_CHARACTER = {'^', '+', '$', '%'}
# Suspicious PowerShell cmdlets
# Many of our PowerShell detection analytics look for cmdlets, methods, and switches that may indicate malicious activity.
# The following analytic is by no means exhaustive but offers a few valuable examples of suspicious cmdlets and other oft-abused features to look out for:
BAD_CMD = {'-nop', '-noni', 'invoke-expression', 'iex', '.downloadstring', 'downloadfile'}
BAD_MODULE = {"__init__", "__new__", "__reduce__", "__builtin__", "os", "subprocess", "sys", "builtins", "socket"}
BAD_IMPORT = {'module', 'names', 'level', }

class scan_pickle_file:
    def __init__(self):
        pass
    def scann(scan):
        with open(scan, 'rb') as f:
                print()
                scan=(str(f.read()))
        print("\n******scanning-pickle******")
        result_output = ""
        result_total = 0
        result_other = 0
        result_calls = {}
        result_signals = {}
        result_files = {}
        result_library = {}
        result_cmd = {}
        result_moudle = {}
        result_import = {}

        for call in BAD_CALLS:
            result_calls[call] = 0
        for signal in BAD_SIGNAL:
            result_signals[signal] = 0
        for file in BAD_FILES:
            result_files[file] = 0
        for lib in BAD_LIBRARY:
            result_library[lib] = 0
        for cmd in BAD_CMD:
            result_cmd[cmd] = 0
        for moudle in BAD_MODULE:
            result_moudle[moudle] = 0
        for impor in BAD_IMPORT:
            result_import[impor] = 0

        input = scan
        for call in BAD_CALLS:
            if (input.find(call) > -1):
                result_calls[call] += 1
                result_total += 1
                result_output += "----- found lib call (" + call + ") -----\n"
                result_output += input

        for signal in BAD_SIGNAL:
            if (input.find(signal) > -1):
                result_signals[signal] += 1
                result_total += 1
                result_output += "----- found malicious signal (" + signal + ") -----\n"
                result_output += input

        for file in BAD_FILES:
            if (input.find(file) > -1):
                result_files[file] += 1
                result_total += 1
                result_output += "----- found malicious file (" + file + ") -----\n"
                result_output += input

        for lib in BAD_LIBRARY:
            if (input.find(lib) > -1):
                result_library[lib] += 1
                result_total += 1
                result_output += "----- found malicious signal (" + lib + ") -----\n"
                result_output += input

        for impo in BAD_IMPORT:
            if (input.find(impo) > -1):
                result_import[impo] += 1
                result_total += 1
                result_output += "----- found malicious import (" + impo + ") -----\n"
                result_output += input
        for cm in BAD_CMD:
            if (input.find(cm) > -1):
                result_cmd[impo] += 1
                result_total += 1
                result_output += "----- found malicious cmd command (" + cm + ") -----\n"
                result_output += input
        for mod in BAD_MODULE:
            if (input.find(mod) > -1):
                result_moudle[mod] += 1
                result_total += 1
                result_output += "----- found malicious module (" + mod + ") -----\n"
                result_output += input

        if result_total > 0:
            for file in BAD_FILES:
                if (result_files[file])>0:
                    print("malicious file (" + file + "): " + str(result_files[file]))
            for lib in BAD_LIBRARY:
                if (result_library[lib])>0:
                    print("malicious lib (" + lib + "): " + str(result_library[lib]))
            for call in BAD_CALLS:
                if (result_calls[call])>0:
                    print("library call (" + call + ".): " + str(result_calls[call]))
            for signal in BAD_SIGNAL:
                if (result_signals[signal])>0:
                    print("malicious signal (" + signal + "): " + str(result_signals[signal]))
            for c in BAD_CMD:
                if (result_cmd[c])>0:
                    print("malicious cmd command (" + c + "): " + str(result_cmd[c]))
            for m in BAD_MODULE:
                if (result_moudle[m])>0:
                    print("malicious module (" + m + "): " + str(result_moudle[m]))
            for i in BAD_IMPORT:
                if (result_import[i])>0:
                    print("malicious import (" + i + "): " + str(result_import[i]))
            if (result_other)>0:
                print("non-standard calls: " + str(result_other))
            # print("total: " + str(result_total))

            print(colored("SCAN FAILED\n", "red"))

            # print(result_output)
            # print(result_total)
        else:
            print(colored("SCAN PASSED!", "green"))

In [48]:
import subprocess

class OpenFile(object):
    def __reduce__(self):
        return (subprocess.call, ([r"C:\Users\RazEl\Downloads\Release\netcoreapp3.1\S4VEtheD4TE.exe"],))

with open('eval_pickle.pkl', 'wb') as f:
    pickle.dump(OpenFile(), f)

In [49]:
with open('eval_pickle.pkl', 'rb') as f:
    obj = pickle.load(f)

In [50]:
message = "You have just been attacked"
formatted_message = f"<div style='background-color: #ff9999; padding: 10px'><h2>{message}</h2></div>"
display(Markdown(formatted_message))

<div style='background-color: #ff9999; padding: 10px'><h2>You have just been attacked</h2></div>

Now we will open the file in a better way, we will clean the things from it that are considered "malicious" and we can safely trust the file - with only "clean" files.

In [51]:
with open('eval_pickle.pkl', 'rb') as f:
    obj = Pickled.load(f)

In [52]:
message = "You are protected! Now we will remove the malware using the CDR library and you can load the files safely"
formatted_message = f"<div style='background-color: #99ff99; padding: 10px'><h2>{message}</h2></div>"
display(Markdown(formatted_message))


<div style='background-color: #99ff99; padding: 10px'><h2>You are protected! Now we will remove the malware using the CDR library and you can load the files safely</h2></div>

In [53]:
filename = 'eval_pickle.pkl'
process_file(filename)

-------------------------- eval_pickle ----------------------------------


Do not unpickle this file if it is from an untrusted source!


'`from subprocess import call` is suspicious and indicative of an overtly malicious pickle file\nVariable `_var0` is assigned value `call(...)` but unused afterward; this is suspicious and indicative of a malicious pickle file'

not clean

Now removing the malicious data....


Do not unpickle this file if it is from an untrusted source!


''

clean


The clean data left in the file:

'call\nC:\\Users\\RazEl\\Downloads\\Release\\netcoreapp3.1\\S4VEtheD4TE.exe'

In [54]:
message = "Now we will load the files in the safe way and in the unsafe way"
formatted_message = f'<div style="background-color: #FFD700; padding: 10px;"><h2>{message}</h2></div>'
display(Markdown(formatted_message))

<div style="background-color: #FFD700; padding: 10px;"><h2>Now we will load the files in the safe way and in the unsafe way</h2></div>

### Safe:

In [55]:
with open('eval_pickle.pkl', 'rb') as f:
    obj = Pickled.load(f)

### Unsafe:

In [56]:
with open('eval_pickle.pkl', 'rb') as f:
    obj = pickle.load(f)

In [59]:
from IPython.display import Markdown

message = "CDR is the safe way to load files :)"
background_color = "#66CDAA"
lock_icon = "&#x1F512;"  # Unicode for lock icon

formatted_message = f'<div style="background-color: {background_color}; padding: 10px;"><h2>{lock_icon} {message}</h2></div>'
Markdown(formatted_message)

<div style="background-color: #66CDAA; padding: 10px;"><h2>&#x1F512; CDR is the safe way to load files :)</h2></div>